<a href="https://colab.research.google.com/github/dat810/tt2_things/blob/main/present_panic/bomb_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prestige Event Bomb Game

Exploring the following:
- Average scissors needed per board
- Average boards completed for the event

So that you are prepared for the RNG 🤣

## Setting Up
Loading libraries

In [4]:
import random
import numpy as np
import ctypes
import matplotlib.pyplot as plt
import pandas as pd
import multiprocessing as mp
import os
import requests

Loading the dataset (as of 7.5.2)

In [5]:
# Define the URL of the dataset and the local filename
url = 'https://github.com/rawrzcookie/TT2_CSV/blob/8212138e8dbe76992552353a4994ed693c5bf737/csv/HolidayEventBombGameLevelInfo.csv'
output_path = 'HolidayEventBombGameLevelInfo.csv'

# Check if the file exists, and download if it doesn't
if not os.path.exists(output_path):
    response = requests.get(url)
    with open(output_path, "wb") as file:
        file.write(response.content)
    print(f"Dataset downloaded and saved to {output_path}")
else:
    print(f"Dataset already exists at {output_path}")

# Load the dataset using pandas
level_info = pd.read_csv('HolidayEventBombGameLevelInfo.csv')

Dataset already exists at HolidayEventBombGameLevelInfo.csv


Defining a Board

In [6]:
class Board:
    def __init__(self,dim,bombs,target,diff_idx):
        self.dim = dim
        self.bombs = bombs
        self.target = target
        self.boxes = self.dim[0]*self.dim[1]

        idx_to_diff = {
            0: 'Easy', 1: 'Hard', 2: 'Impossible'
        }
        self.diff = idx_to_diff.get(diff_idx)


    def __str__(self):
        return (
            f"Board size: {self.dim[0]}x{self.dim[1]}\n"
            f"Bombs count: {self.bombs}\n"
            f"Streak needed: {self.target}\n"
            f"Difficulty: {self.diff}"
        )

## Average Scissors Needed per Board

## Average Boards Completed in an Event

## Notebook Information
- Notebook content created by dat
- Dataset from rawrzcookie, who also fetched the CSV files here: https://github.com/rawrzcookie/TT2_CSV